# Setup Environment

In [ ]:
# !pip install ../input/fastai2-offline/timm-0.2.1-py3-none-any.whl ../input/fastai2-offline/wwf-0.0.3-py3-none-any.whl -q

In [ ]:
import pandas as pd
from fastai.vision.all import *
# from wwf.vision.timm import *
# from fastai.callback.cutmix import *

import os

# Import Train Data

In [ ]:
path = Path('../input/cassava-leaf-disease-classification')

# 2019 and 2020 merge data path
# data_path = Path('../input/cassava-datasetv2/cassava_datasetv2')
data_path = Path('../input/cassava-leaf-disease-merged')

In [ ]:
# train_df = pd.read_csv(data_path/'train.csv')
train_df = pd.read_csv(data_path/'merged.csv')
train_df

Remove the duplicate image according to this [discussion](https://www.kaggle.com/c/cassava-leaf-disease-classification/discussion/198202)
* '1562043567.jpg' and '3551135685.jpg' (mislabel)
* '2252529694.jpg' and '911861181.jpg' (duplicate)


In [ ]:
# train_df = train_df[~train_df['image_id'].isin(['1562043567.jpg', '3551135685.jpg', '2252529694.jpg'])]

# Create Dataloaders

In [ ]:
def get_x(r):
#     return data_path/'train_images'/r['image_id']
    return data_path/'train'/r['image_id']

def get_y(r):
    return r['label']

In [ ]:
def get_data(size=224,bs=128,data_df=train_df):
    dblock = DataBlock(blocks=(ImageBlock, CategoryBlock),
                       splitter=RandomSplitter(seed=42),
                       get_x=get_x, 
                       get_y=get_y,
                       item_tfms = RandomResizedCrop(size=size, min_scale=0.75, ratio=(1.,1.)),
                       batch_tfms = [*aug_transforms(size=size, flip_vert=True),Normalize.from_stats(*imagenet_stats)]
                      )
    return dblock.dataloaders(data_df,bs=bs)

Get dataloader and show the data

In [ ]:
dls = get_data(size=112)
dls.show_batch()

# Create learner

In [ ]:
if not os.path.exists('/root/.cache/torch/hub/checkpoints/'):
        os.makedirs('/root/.cache/torch/hub/checkpoints/')
!cp '../input/resnet50/resnet50.pth' '/root/.cache/torch/hub/checkpoints/resnet50-19c8e357.pth'
# !cp '../input/timm-pretrained-efficientnet/efficientnet/efficientnet_b0_ra-3dd342df.pth' '/root/.cache/torch/hub/checkpoints/efficientnet_b0_ra-3dd342df.pth'
# !cp '../input/timm-pretrained-efficientnet/efficientnet/efficientnet_b3_ra2-cf984f9c.pth' '/root/.cache/torch/hub/checkpoints/efficientnet_b3_ra2-cf984f9c.pth'
# !cp '../input/timm-pretrained-efficientnet/efficientnet/tf_efficientnet_b3_ns-9d44bf68.pth' '/root/.cache/torch/hub/checkpoints/tf_efficientnet_b3_ns-9d44bf68.pth'
# !cp '../input/timm-pretrained-resnet/resnet/seresnext26d_32x4d-80fa48a3.pth' '/root/.cache/torch/hub/checkpoints/seresnext26d_32x4d-80fa48a3.pth'
# !cp '../input/timm-pretrained-resnet/resnet/wide_resnet50_2-95faca4d.pth' '/root/.cache/torch/hub/checkpoints/wide_resnet50_2-95faca4d.pth'
# !cp '../input/timm-pretrained-resnet/resnet/resnext50_32x4d_ra-d733960d.pth' '/root/.cache/torch/hub/checkpoints/resnext50_32x4d_ra-d733960d.pth'

In [ ]:
loss_func = LabelSmoothingCrossEntropy()
cbs = MixUp

In [ ]:
learn = cnn_learner(dls, resnet50, loss_func=loss_func, metrics=accuracy, cbs=cbs).to_native_fp16()

# learn = timm_learner(dls, 'efficientnet_b3', loss_func=loss_func, metrics=accuracy, cbs=cbs).to_native_fp16()

In [ ]:
learn.lr_find()

In [ ]:
learn.fine_tune(20, base_lr=1e-2 , freeze_epochs=8)

resize to 224

In [ ]:
learn.dls = get_data()
learn.dls.show_batch()

In [ ]:
learn.lr_find()

In [ ]:
learn.fine_tune(20, freeze_epochs=8)

In [ ]:
learn = learn.to_native_fp32()

# Make Submission file

In [ ]:
submission_df = pd.read_csv(path/'sample_submission.csv')
submission_df

prediction using test time augmentation(TTA)

In [ ]:
test_data_path = submission_df['image_id'].apply(lambda x: path/'test_images'/x)
tst_dl = learn.dls.test_dl(test_data_path)
predictions = learn.tta(dl = tst_dl, n=10, beta=0)

submission_df['label'] = np.argmax(predictions[0],axis=1)
submission_df

In [ ]:
submission_df.to_csv('submission.csv',index=False)